<a href="https://colab.research.google.com/github/bansarithummar/Recommendation-system/blob/main/Collaborative%20Filtering/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Dataset: http://files.grouplens.org/datasets/movielens/

In [36]:
from google.colab import files
uploaded = files.upload()

Saving ml-latest-small.zip to ml-latest-small.zip


In [37]:
import zipfile
import os

zip_file = 'ml-latest-small.zip'

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('data')

extracted_files = os.listdir('data')
print(extracted_files)

['ml-latest-small']


In [38]:
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [39]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)


The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [40]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [41]:

# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [42]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")


Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [47]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm
from torch import nn
from torch.utils.data import DataLoader, Dataset

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=8):
        super(MatrixFactorization, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = nn.Embedding(n_items, n_factors, sparse=True)
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users) * self.item_factors(items)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [52]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

class Loader(Dataset):
    def __init__(self, ratings_df):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        # Producing new continuous IDs for users and movies
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])

        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x, dtype=torch.long), torch.tensor(self.y, dtype=torch.float32)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

# Create DataLoader
train_set = Loader(ratings_df)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)


In [53]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader(ratings_df)
print("Data loader created successfully.")
print("First batch of data:", next(iter(train_set)))


Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8, sparse=True)
  (item_factors): Embedding(9724, 8, sparse=True)
)
user_factors.weight tensor([[0.0346, 0.0065, 0.0089,  ..., 0.0465, 0.0142, 0.0087],
        [0.0410, 0.0060, 0.0460,  ..., 0.0035, 0.0185, 0.0285],
        [0.0458, 0.0010, 0.0218,  ..., 0.0468, 0.0140, 0.0410],
        ...,
        [0.0303, 0.0207, 0.0009,  ..., 0.0209, 0.0015, 0.0015],
        [0.0433, 0.0459, 0.0154,  ..., 0.0462, 0.0390, 0.0487],
        [0.0394, 0.0054, 0.0435,  ..., 0.0140, 0.0482, 0.0195]])
item_factors.weight tensor([[0.0029, 0.0202, 0.0161,  ..., 0.0035, 0.0255, 0.0113],
        [0.0422, 0.0440, 0.0073,  ..., 0.0285, 0.0488, 0.0444],
        [0.0290, 0.0227, 0.0014,  ..., 0.0492, 0.0451, 0.0087],
        ...,
        [0.0132, 0.0403, 0.0025,  ..., 0.0174, 0.0238, 0.0058],
        [0.0437, 0.0270, 0.0221,  ..., 0.0243, 0.0051, 0.0449],
        [0.0041, 0.0175, 0.0061,  ..., 0.0316, 0.0292, 0.0399]])
Data loader create

In [56]:
from tqdm import tqdm
optimizer = torch.optim.SparseAdam(model.parameters(), lr=1e-3)

for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if torch.cuda.is_available():
            x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

  1%|          | 1/128 [00:03<06:21,  3.00s/it]

iter #0 Loss: 12.013407802823837


  2%|▏         | 2/128 [00:05<05:27,  2.60s/it]

iter #1 Loss: 6.937140610919991


  2%|▏         | 3/128 [00:07<05:09,  2.47s/it]

...

iter #125 Loss: 0.4853124328449293


 99%|█████████▉| 127/128 [05:11<00:02,  2.41s/it]

iter #126 Loss: 0.4833949909446203


100%|██████████| 128/128 [05:13<00:00,  2.45s/it]

iter #127 Loss: 0.4813890685965567


In [57]:
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.5710,  1.4464,  1.5014,  ...,  2.1994,  1.4891,  1.3978],
        [ 1.3227,  1.3260,  1.2967,  ...,  1.1947,  1.3056,  1.1760],
        [ 0.8365,  0.7280,  0.7439,  ...,  0.9421,  0.6307,  0.9601],
        ...,
        [ 0.9910, -0.5364, -0.5619,  ...,  2.5521,  0.2769,  1.8829],
        [ 1.1453,  1.3251,  1.2447,  ...,  1.0617,  1.0863,  1.2091],
        [ 1.5815,  1.8342, -0.8168,  ...,  0.3223,  3.4216,  1.7629]],
       device='cuda:0')
item_factors.weight tensor([[0.3770, 0.3963, 0.3489,  ..., 0.5031, 0.5135, 0.3426],
        [0.0861, 0.2576, 0.4545,  ..., 0.5006, 0.3859, 0.4798],
        [0.5174, 0.2763, 0.2826,  ..., 0.3982, 0.4691, 0.2149],
        ...,
        [0.2708, 0.3025, 0.2564,  ..., 0.2723, 0.2884, 0.2678],
        [0.3392, 0.3278, 0.2994,  ..., 0.3063, 0.3187, 0.3467],
        [0.2971, 0.3158, 0.2826,  ..., 0.3123, 0.3392, 0.3391]],
       device='cuda:0')


In [58]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [60]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [61]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [62]:

'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Ace Ventura: When Nature Calls (1995)
	 Nutty Professor, The (1996)
	 Honey, I Shrunk the Kids (1989)
	 Blair Witch Project, The (1999)
	 Coneheads (1993)
	 Judge Dredd (1995)
	 Hot Shots! Part Deux (1993)
	 Beverly Hills Cop III (1994)
	 Cable Guy, The (1996)
	 Arachnophobia (1990)
Cluster #1
	 Shawshank Redemption, The (1994)
	 Pulp Fiction (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Braveheart (1995)
	 Terminator 2: Judgment Day (1991)
	 Schindler's List (1993)
	 Fight Club (1999)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
Cluster #2
	 Forrest Gump (1994)
	 Toy Story (1995)
	 Apollo 13 (1995)
	 Aladdin (1992)
	 Shrek (2001)
	 Dances with Wolves (1990)
	 Truman Show, The (1998)
	 Minority Report (2002)
	 Ocean's Eleven (2001)
	 Catch Me If You Can (2002)
Cluster #3
	 Wild Wild West (1999)
	 Batman & Robin (1997)
	 Godzilla (1998)
	 I Know What You Did Last Summer (1997)
	 Anaconda (1997)
	 Rich